# Train a JointVAE model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch

use_cuda = torch.cuda.is_available()
use_cuda

True

In [25]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # change to your device

#### Prepare data list

#### Create list of image paths

In [5]:
#loadable_dresses = list(np.loadtxt('data/loadable_women_primary_dress.csv',delimiter=',',skiprows=1,dtype='str'))

In [6]:
#print(len(loadable_dresses))

In [18]:
#loadable_dresses[:5]

In [9]:
!ls /workspace/mnt/crucial_2TB/111_Extra_Data/Macys/

imgs224_1000  imgs260_3150  xtrain3150_260260.csv  xtrainonly1000.csv


['/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_2658.png',
 '/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_986.png',
 '/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_693.png']

In [11]:
len(files)

3151

#### Split into train and test set

In [13]:
#image_paths_train = loadable_dresses[:88800]
#image_paths_test = loadable_dresses[88800:-124]
image_paths_train = files[:2800]
image_paths_test = files[2800:-1]


print(f"Number of train image paths: {len(image_paths_train):,d}")
print(f"Number of test image paths: {len(image_paths_test):,d}")
print()
print("Sample paths:")
print(image_paths_train[0])
print(image_paths_train[-1])
print(image_paths_test[0])
print(image_paths_test[-1])

Number of train image paths: 2,800
Number of test image paths: 350

Sample paths:
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_2658.png
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_131.png
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_2921.png
/workspace/mnt/crucial_2TB/111_Extra_Data/Macys/imgs260_3150/img_2446.png


In [27]:
from torchvision import transforms
#from utils.dataloaders_custom import get_imagelist_dataloader, ImageListDataset
from utils.dataloader_tools import get_imagelist_dataloader, ImageListDataset

BATCH_SIZE = 50

composed = transforms.Compose([transforms.CenterCrop((256,256)),transforms.Resize((256,256)),transforms.ToTensor()])

# convert rgb is for the cv2 loaded images that I've got in this dir
conv_rgb = True
train_dataset = ImageListDataset(image_paths_train, cut_from='top', cut_amount=256, transform=composed, convert_rgb=conv_rgb)
test_dataset = ImageListDataset(image_paths_test, cut_from='top', cut_amount=256, transform=composed, convert_rgb=conv_rgb)

train_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=train_dataset)
test_loader = get_imagelist_dataloader(batch_size=BATCH_SIZE, dataset_object=test_dataset)

### Define latent distribution of the model

In [28]:
# Latent distribution will be joint distribution of 10 gaussian normal distributions
# and one 10 dimensional Gumbel Softmax distribution
latent_spec = {'cont': 10,
               'disc': [10]}

### Build a model

In [29]:
!ls jointvae

__init__.py	       models_128_xstyle_int_debug.py
__pycache__	       models_128_xstyle_int_nd.py
decoder_scratchpad.py  models_256_xstyle_int_nd.py
encoder_scratchpad.py  models_64_xstyle.py
models.py	       models_64_xstyle_finished_v1.py
models_128_v1.py       models_64_xstyle_int_debug.py
models_128_v2.py       models_64_xstyle_int_nd.py
models_128_v3.py       training.py
models_128_v4.py       training_debug.py


In [30]:
from jointvae.models_256_xstyle_int_nd import VAE

#model = VAE(latent_spec=latent_spec, img_size=(3, 260, 260), use_cuda=use_cuda)
model = VAE(latent_spec=latent_spec, img_size=(3, 256, 256), use_cuda=use_cuda)

In [31]:
#print(model)

### Train the model

In [36]:
from torch import optim

# Build optimizer
optimizer = optim.Adam(model.parameters(), lr=9e-5, amsgrad=True) # added amsgrad # orig lr 5e-4

In [37]:
from jointvae.training import Trainer
#from jointvae.training_debug import Trainer

# Define the capacities
# Continuous channels
cont_capacity = [0.0, 5.2, 30000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0
# Discrete channels
disc_capacity = [0.0, 5.2, 30000, 30.0]  # Starting at a capacity of 0.0, increase this to 5.0
                                         # over 25000 iterations with a gamma of 30.0

# Build a trainer
trainer = Trainer(model, optimizer,
                  cont_capacity=cont_capacity,
                  disc_capacity=disc_capacity,
                 use_cuda=use_cuda)

#### Initialize visualizer

In [38]:
#from viz.visualize import Visualizer
from viz.visualize import Visualizer

viz = Visualizer(model)
viz.save_images = False # needed to add this so it returns a tensor

In [ ]:
# Train model for 10 epochs
# Note this should really be a 100 epochs and trained on a GPU, but this is just to demo

trainer.train(train_loader, epochs=25000, save_training_gif=None)

0/2800	Loss: 45225.793
2500/2800	Loss: 45244.248
Epoch: 1 Average loss: 45243.74
0/2800	Loss: 45274.816
2500/2800	Loss: 45240.053
Epoch: 2 Average loss: 45240.92
0/2800	Loss: 45243.957
2500/2800	Loss: 45236.826
Epoch: 3 Average loss: 45238.38
0/2800	Loss: 45207.934
2500/2800	Loss: 45235.634
Epoch: 4 Average loss: 45236.15
0/2800	Loss: 45229.992
2500/2800	Loss: 45233.288
Epoch: 5 Average loss: 45233.86
0/2800	Loss: 45214.027
2500/2800	Loss: 45233.093
Epoch: 6 Average loss: 45232.89
0/2800	Loss: 45247.801
2500/2800	Loss: 45231.141
Epoch: 7 Average loss: 45230.80
0/2800	Loss: 45203.926
2500/2800	Loss: 45230.323
Epoch: 8 Average loss: 45229.94
0/2800	Loss: 45280.660
2500/2800	Loss: 45228.338
Epoch: 9 Average loss: 45228.56
0/2800	Loss: 45237.027
2500/2800	Loss: 45229.111
Epoch: 10 Average loss: 45228.37
0/2800	Loss: 45237.973
2500/2800	Loss: 45225.617
Epoch: 11 Average loss: 45226.07
0/2800	Loss: 45231.691
2500/2800	Loss: 45226.150
Epoch: 12 Average loss: 45225.48
0/2800	Loss: 45260.262
25

2500/2800	Loss: 44907.125
Epoch: 101 Average loss: 44907.57
0/2800	Loss: 44963.902
2500/2800	Loss: 44908.653
Epoch: 102 Average loss: 44908.27
0/2800	Loss: 44899.055
2500/2800	Loss: 44909.702
Epoch: 103 Average loss: 44908.20
0/2800	Loss: 44975.656
2500/2800	Loss: 44908.031
Epoch: 104 Average loss: 44906.60
0/2800	Loss: 44886.383
2500/2800	Loss: 44903.751
Epoch: 105 Average loss: 44905.00
0/2800	Loss: 44952.078
2500/2800	Loss: 44900.064
Epoch: 106 Average loss: 44905.65
0/2800	Loss: 44939.633
2500/2800	Loss: 44905.567
Epoch: 107 Average loss: 44903.70
0/2800	Loss: 44873.602
2500/2800	Loss: 44901.740
Epoch: 108 Average loss: 44900.63
0/2800	Loss: 44959.355
2500/2800	Loss: 44892.917
Epoch: 109 Average loss: 44900.13
0/2800	Loss: 44858.871
2500/2800	Loss: 44904.365
Epoch: 110 Average loss: 44898.31
0/2800	Loss: 44967.312
2500/2800	Loss: 44890.267
Epoch: 111 Average loss: 44897.81
0/2800	Loss: 44921.980
2500/2800	Loss: 44897.404
Epoch: 112 Average loss: 44897.02
0/2800	Loss: 44920.625
2500

0/2800	Loss: 44871.852
2500/2800	Loss: 44817.395
Epoch: 200 Average loss: 44822.05
0/2800	Loss: 44732.586
2500/2800	Loss: 44823.352
Epoch: 201 Average loss: 44820.29
0/2800	Loss: 44813.496
2500/2800	Loss: 44814.109
Epoch: 202 Average loss: 44819.97
0/2800	Loss: 44834.637
2500/2800	Loss: 44813.709
Epoch: 203 Average loss: 44813.24
0/2800	Loss: 44796.027
2500/2800	Loss: 44814.051
Epoch: 204 Average loss: 44813.67
0/2800	Loss: 44743.148
2500/2800	Loss: 44818.295
Epoch: 205 Average loss: 44812.14
0/2800	Loss: 44826.586
2500/2800	Loss: 44817.140
Epoch: 206 Average loss: 44811.50
0/2800	Loss: 44865.195
2500/2800	Loss: 44810.033
Epoch: 207 Average loss: 44808.95
0/2800	Loss: 44785.969
2500/2800	Loss: 44805.723
Epoch: 208 Average loss: 44807.72
0/2800	Loss: 44830.383
2500/2800	Loss: 44809.344
Epoch: 209 Average loss: 44804.68
0/2800	Loss: 44889.613
2500/2800	Loss: 44799.238
Epoch: 210 Average loss: 44805.35
0/2800	Loss: 44892.070
2500/2800	Loss: 44795.015
Epoch: 211 Average loss: 44804.53
0/28

0/2800	Loss: 44842.117
2500/2800	Loss: 44746.722
Epoch: 299 Average loss: 44748.28
0/2800	Loss: 44727.203
2500/2800	Loss: 44752.432
Epoch: 300 Average loss: 44750.09
0/2800	Loss: 44647.598
2500/2800	Loss: 44744.813
Epoch: 301 Average loss: 44746.78
0/2800	Loss: 44698.523
2500/2800	Loss: 44749.338
Epoch: 302 Average loss: 44747.83
0/2800	Loss: 44591.277
2500/2800	Loss: 44750.846
Epoch: 303 Average loss: 44748.23
0/2800	Loss: 44807.609
2500/2800	Loss: 44741.389
Epoch: 304 Average loss: 44745.26
0/2800	Loss: 44735.012
2500/2800	Loss: 44737.826
Epoch: 305 Average loss: 44741.68
0/2800	Loss: 44747.727
2500/2800	Loss: 44750.469
Epoch: 306 Average loss: 44742.97
0/2800	Loss: 44773.145
2500/2800	Loss: 44745.923
Epoch: 307 Average loss: 44745.21
0/2800	Loss: 44817.961
2500/2800	Loss: 44737.246
Epoch: 308 Average loss: 44743.99
0/2800	Loss: 44573.512
2500/2800	Loss: 44742.799
Epoch: 309 Average loss: 44739.29
0/2800	Loss: 44682.121
2500/2800	Loss: 44747.531
Epoch: 310 Average loss: 44741.82
0/28

0/2800	Loss: 44623.789
2500/2800	Loss: 44692.526
Epoch: 398 Average loss: 44693.17
0/2800	Loss: 44823.203
2500/2800	Loss: 44688.481
Epoch: 399 Average loss: 44693.54
0/2800	Loss: 44567.266
2500/2800	Loss: 44694.964
Epoch: 400 Average loss: 44693.96
0/2800	Loss: 44680.945
2500/2800	Loss: 44693.624
Epoch: 401 Average loss: 44693.67
0/2800	Loss: 44713.637
2500/2800	Loss: 44691.882
Epoch: 402 Average loss: 44691.97
0/2800	Loss: 44772.344
2500/2800	Loss: 44684.957
Epoch: 403 Average loss: 44689.99
0/2800	Loss: 44636.215
2500/2800	Loss: 44693.312
Epoch: 404 Average loss: 44690.98
0/2800	Loss: 44554.715
2500/2800	Loss: 44704.857
Epoch: 405 Average loss: 44690.41
0/2800	Loss: 44434.805
2500/2800	Loss: 44690.701
Epoch: 406 Average loss: 44691.37
0/2800	Loss: 44756.617
2500/2800	Loss: 44693.282
Epoch: 407 Average loss: 44686.07
0/2800	Loss: 44718.090
2500/2800	Loss: 44682.435
Epoch: 408 Average loss: 44687.31
0/2800	Loss: 44759.465
2500/2800	Loss: 44681.094
Epoch: 409 Average loss: 44687.06
0/28

...

Epoch: 4996 Average loss: 44409.20
0/2800	Loss: 44422.121
2500/2800	Loss: 44411.691
Epoch: 4997 Average loss: 44410.83
0/2800	Loss: 44365.367
2500/2800	Loss: 44412.035
Epoch: 4998 Average loss: 44408.51
0/2800	Loss: 44169.469
2500/2800	Loss: 44405.945
Epoch: 4999 Average loss: 44406.87
0/2800	Loss: 44363.465
2500/2800	Loss: 44418.583
Epoch: 5000 Average loss: 44410.57
0/2800	Loss: 44380.598
2500/2800	Loss: 44410.012
Epoch: 5001 Average loss: 44409.90
0/2800	Loss: 44563.676
2500/2800	Loss: 44398.502
Epoch: 5002 Average loss: 44406.12
0/2800	Loss: 44262.750
2500/2800	Loss: 44411.164
Epoch: 5003 Average loss: 44406.87
0/2800	Loss: 44266.172
2500/2800	Loss: 44412.912
Epoch: 5004 Average loss: 44412.06
0/2800	Loss: 44519.637
2500/2800	Loss: 44409.575
Epoch: 5005 Average loss: 44409.36
0/2800	Loss: 44488.203
2500/2800	Loss: 44406.192
Epoch: 5006 Average loss: 44409.15
0/2800	Loss: 44238.395
2500/2800	Loss: 44408.926
Epoch: 5007 Average loss: 44408.42
0/2800	Loss: 44428.043
2500/2800	Loss: 44

2500/2800	Loss: 44407.202
Epoch: 5094 Average loss: 44407.43
0/2800	Loss: 44188.473
2500/2800	Loss: 44415.197
Epoch: 5095 Average loss: 44408.00
0/2800	Loss: 44506.398
2500/2800	Loss: 44403.647
Epoch: 5096 Average loss: 44407.08
0/2800	Loss: 44518.746
2500/2800	Loss: 44411.395
Epoch: 5097 Average loss: 44409.84
0/2800	Loss: 44368.637
2500/2800	Loss: 44406.183
Epoch: 5098 Average loss: 44410.32
0/2800	Loss: 44335.555
2500/2800	Loss: 44414.092
Epoch: 5099 Average loss: 44409.00
0/2800	Loss: 44362.855
2500/2800	Loss: 44407.661
Epoch: 5100 Average loss: 44405.63
0/2800	Loss: 44409.734
2500/2800	Loss: 44412.814
Epoch: 5101 Average loss: 44408.65
0/2800	Loss: 44655.270
2500/2800	Loss: 44412.193
Epoch: 5102 Average loss: 44409.24
0/2800	Loss: 44332.023
2500/2800	Loss: 44414.849
Epoch: 5103 Average loss: 44408.98
0/2800	Loss: 44325.516
2500/2800	Loss: 44401.894
Epoch: 5104 Average loss: 44407.61
0/2800	Loss: 44433.449
2500/2800	Loss: 44406.637
Epoch: 5105 Average loss: 44408.47
0/2800	Loss: 44

0/2800	Loss: 44233.742
2500/2800	Loss: 44415.909
Epoch: 5192 Average loss: 44404.60
0/2800	Loss: 44346.203
2500/2800	Loss: 44421.696
Epoch: 5193 Average loss: 44408.00
0/2800	Loss: 44473.828
2500/2800	Loss: 44401.284
Epoch: 5194 Average loss: 44406.24
0/2800	Loss: 44489.242
2500/2800	Loss: 44405.857
Epoch: 5195 Average loss: 44407.68
0/2800	Loss: 44407.738
2500/2800	Loss: 44407.754
Epoch: 5196 Average loss: 44407.08
0/2800	Loss: 44547.836
2500/2800	Loss: 44405.909
Epoch: 5197 Average loss: 44407.70
0/2800	Loss: 44423.012
2500/2800	Loss: 44404.923
Epoch: 5198 Average loss: 44405.74
0/2800	Loss: 44440.496
2500/2800	Loss: 44409.232
Epoch: 5199 Average loss: 44406.22
0/2800	Loss: 44385.855
2500/2800	Loss: 44403.738
Epoch: 5200 Average loss: 44403.80
0/2800	Loss: 44605.887
2500/2800	Loss: 44406.048
Epoch: 5201 Average loss: 44404.16
0/2800	Loss: 44460.156
2500/2800	Loss: 44392.160
Epoch: 5202 Average loss: 44405.98
0/2800	Loss: 44523.793
2500/2800	Loss: 44400.031
Epoch: 5203 Average loss: 4

Epoch: 5289 Average loss: 44405.39
0/2800	Loss: 44542.082
2500/2800	Loss: 44399.371
Epoch: 5290 Average loss: 44406.42
0/2800	Loss: 44372.344
2500/2800	Loss: 44415.121
Epoch: 5291 Average loss: 44406.32
0/2800	Loss: 44116.648
2500/2800	Loss: 44412.180
Epoch: 5292 Average loss: 44405.75
0/2800	Loss: 44411.578
2500/2800	Loss: 44406.048
Epoch: 5293 Average loss: 44404.08
0/2800	Loss: 44521.578
2500/2800	Loss: 44414.553
Epoch: 5294 Average loss: 44405.44
0/2800	Loss: 44569.301
2500/2800	Loss: 44406.092
Epoch: 5295 Average loss: 44404.72
0/2800	Loss: 44263.754
2500/2800	Loss: 44407.870
Epoch: 5296 Average loss: 44403.04
0/2800	Loss: 44493.633
2500/2800	Loss: 44403.575
Epoch: 5297 Average loss: 44403.60
0/2800	Loss: 44394.613
2500/2800	Loss: 44397.604
Epoch: 5298 Average loss: 44404.17
0/2800	Loss: 44413.570
2500/2800	Loss: 44395.626
Epoch: 5299 Average loss: 44404.46
0/2800	Loss: 44253.996
2500/2800	Loss: 44399.975
Epoch: 5300 Average loss: 44404.70
0/2800	Loss: 44499.504
2500/2800	Loss: 44

2500/2800	Loss: 44393.430
Epoch: 5387 Average loss: 44404.36
0/2800	Loss: 44307.781
2500/2800	Loss: 44409.034
Epoch: 5388 Average loss: 44403.71
0/2800	Loss: 44494.445
2500/2800	Loss: 44411.906
Epoch: 5389 Average loss: 44402.11
0/2800	Loss: 44357.566
2500/2800	Loss: 44403.018
Epoch: 5390 Average loss: 44401.86
0/2800	Loss: 44502.234
2500/2800	Loss: 44405.551
Epoch: 5391 Average loss: 44404.63
0/2800	Loss: 44371.145
2500/2800	Loss: 44406.602
Epoch: 5392 Average loss: 44403.50
0/2800	Loss: 44405.938
2500/2800	Loss: 44403.226
Epoch: 5393 Average loss: 44404.68
0/2800	Loss: 44619.629
2500/2800	Loss: 44399.197
Epoch: 5394 Average loss: 44404.48
0/2800	Loss: 44080.773
2500/2800	Loss: 44402.981
Epoch: 5395 Average loss: 44402.74
0/2800	Loss: 44366.426
2500/2800	Loss: 44404.705
Epoch: 5396 Average loss: 44408.41
0/2800	Loss: 44484.449
2500/2800	Loss: 44403.070
Epoch: 5397 Average loss: 44402.90
0/2800	Loss: 44491.453
2500/2800	Loss: 44402.960
Epoch: 5398 Average loss: 44402.81
0/2800	Loss: 44

In [ ]:
print('hi')

In [ ]:
# Get a batch of data
for batch, labels in test_loader:
    break
print("batch: ",type(batch),batch.shape)
type(viz.reconstructions(batch))

### Visualize

In [ ]:
# Plot reconstructions
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Get a batch of data
for batch, labels in test_loader:
    break

# Reconstruct data using Joint-VAE model
recon = viz.reconstructions(batch)

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(recon.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_reconstructions.png",dpi=200)

In [ ]:
# Plot samples
samples = viz.samples()

plt.figure(figsize=(26,26))
plt.imshow(np.rot90(np.transpose(samples.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_samples.png",dpi=200)

#### Traverses all latent dimensions one by one and plots a grid of images where each row corresponds to a latent traversal of one latent dimension

In [ ]:
# Plot all traversals
traversals = viz.all_latent_traversals(size=10)

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_all_traversals_n20.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=2, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(20,20))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_traversals2100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=1, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
plt.imshow(traversals.numpy()[0, :, :]);
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_traversals1100.png",dpi=200)

In [ ]:
# Plot a grid of some traversals
traversals = viz.latent_traversal_grid(cont_idx=9, cont_axis=1, disc_idx=0, disc_axis=0, size=(10, 10))

plt.figure(figsize=(16,16))
plt.imshow(np.rot90(np.transpose(traversals.numpy(),(2,1,0)),k=3));
plt.savefig("sample_images/home/256/xint256_e25k_b50_c10d10_gam30_traversals9100.png",dpi=200)

In [ ]:
!ls

### Save Model

In [ ]:
model_name = "xint256_e25k_b50_c10d10_gam30.pth"

In [ ]:
torch.save(model.state_dict(),"trained_models" + "statedict_" + model_name) # save state dict
#torch.save(model, model_name) # save full model

In [ ]:
print("Done training: ",model_name)

#### Restore Model from State Dict

In [ ]:
sd_model = VAE(latent_spec=latent_spec, img_size=(3, 64, 64))
sd_model.load_state_dict(torch.load("statedict_" + model_name))

#### Restore Full Model
* Note in this case the serialized data is bound to the specific classes and exact directory strucutre used.

In [ ]:
full_model = torch.load(model_name)

In [ ]:
type(full_model)

In [ ]:
type(sd_model)

In [ ]:
!ls